In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from functools import reduce

from src.data import make_dataset
from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s

In [3]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv", infer_schema_length=10000, null_values="Extranjero"
)

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [5]:
display(metricas.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2014,113180,"""Hospital El Pi…","""N840""",10,14,10,0
2017,110100,"""Hospital San J…","""C851""",3,31,0,1
2006,129100,"""Hospital Dr. M…","""I848""",2,8,2,0
2015,111200,"""Hospital Clíni…","""K640""",3,11,1,0
2010,106214,"""Clínica Valpar…","""T024""",1,6,1,0
2006,118202,"""Clínica de la …","""K610""",10,15,10,0
2014,125100,"""Hospital Regio…","""D390""",1,1,1,0
2014,115107,"""Hospital San J…","""B018""",2,12,0,0
2015,111200,"""Hospital Clíni…","""M130""",3,28,0,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
hospitales - diagnosticos distintos.

Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)

## Análisis de Estratos

In [6]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_DEL_TORAX)

In [7]:
cie = build_features.leer_cie()

In [14]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

In [15]:
rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

In [9]:


ranking_nacional_con_cie = build_features.agregar_ranking_estratos()

ranking_nacional_con_cie_pandas = ranking_nacional_con_cie.to_pandas()
ranking_torax = ranking_nacional_con_cie_pandas.query(
    "ESTABLECIMIENTO_SALUD == @HOSPITAL_DEL_TORAX"
)
DIAGS_TORAX = ranking_torax["DIAG1"].unique()

ranking_nacional_diags_torax = ranking_nacional_con_cie_pandas[
    ranking_nacional_con_cie_pandas["DIAG1"].isin(DIAGS_TORAX)
].query("ANO_EGRESO > 2002")

In [9]:
# ranking_nacional_con_cie_pandas.to_csv(
#     "../data/interim/ranking_nacional_egresos.csv",
#     sep=";",
#     decimal=".",
#     encoding="latin-1",
#     index=False
# )

# ranking_torax.to_csv(
#     "../data/interim/ranking_torax_egresos.csv",
#     sep=";",
#     decimal=".",
#     encoding="latin-1",
#     index=False
# )

# ranking_nacional_diags_torax.to_excel(
#     "../data/interim/ranking_nacional_diags_torax_post_2001.xlsx",
#     index=False
# )

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.

In [10]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [11]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()

In [12]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION


dict_resultados_sobre_14 = build_features.obtener_resumen_por_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

ranking_nacional_sobre_14 = reduce(
    build_features.left_join_consecutivo, dict_resultados_sobre_14.values()
)
ranking_nacional_con_cie_sobre_14 = ranking_nacional_sobre_14.join(
    cie, how="left", on="DIAG1"
).select(ORDEN_COLS)

ranking_nacional_con_cie_pandas_sobre_14 = ranking_nacional_con_cie_sobre_14.to_pandas()
ranking_torax_sobre_14 = ranking_nacional_con_cie_pandas_sobre_14.query(
    "ESTABLECIMIENTO_SALUD == @HOSPITAL_DEL_TORAX"
)
DIAGS_TORAX = ranking_torax_sobre_14["DIAG1"].unique()

ranking_nacional_diags_torax_sobre_14 = ranking_nacional_con_cie_pandas_sobre_14[
    ranking_nacional_con_cie_pandas_sobre_14["DIAG1"].isin(DIAGS_TORAX)
].query("ANO_EGRESO > 2002")

In [26]:
ranking_nacional_diags_torax_sobre_14.to_csv(
    "../data/interim/ranking_nacional_sobre_14.csv",
    index=False,
    encoding="latin-1",
    sep=";",
    decimal=".",
)

In [27]:
requerimiento = ranking_nacional_diags_torax_sobre_14[
    ranking_nacional_diags_torax_sobre_14["DIAG1"].isin(["Q230", "Q211", "Q231"])
]
requerimiento = requerimiento.query("ANO_EGRESO == 2019")

In [28]:
requerimiento.to_csv(
    "../data/interim/congenitas_sobre_14.csv",
    index=False,
    encoding="latin-1",
    sep=";",
    decimal=".",
)

In [29]:
requerimiento_control = ranking_nacional_diags_torax[
    ranking_nacional_diags_torax["DIAG1"].isin(["Q230", "Q211", "Q231"])
].query("ANO_EGRESO == 2019")

requerimiento_control.to_csv(
    "../data/interim/congenitas.csv",
    index=False,
    encoding="latin-1",
    sep=";",
    decimal=".",
)